In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import networkx as nx
import osmnx as ox
import geopandas as gpd
import networkx as nx

In [ ]:
#load in the hospital sites data to be used to score potential viable locations
hospitals = pd.read_csv('data/cambridge_uni_hospital_sites.csv')
hospitals.head(2)

In [ ]:

G = ox.graph.graph_from_place('Cambridge', simplify = True, network_type = 'walk')

# The following code can be run to decrease the time required to generate OSMnx network graphs.
# If a node cannot access at least 10% of other nodes it is removed from the network
#---------------------------------------------------------------------------------------

'''
remove_list = []
num_nodes = len(G.nodes)
for node in G.nodes:  
     reach = len(nx.descendants(G, node))
     if reach < num_nodes / 10:
         remove_list.append(node)
for node in remove_list:
     G.remove_node(node)
'''
#---------------------------------------------------------------------------------------
ox.plot_graph(G, figsize = (10,10));

In [ ]:
nodes, edges = ox.graph_to_gdfs(G)
nodes

In [ ]:
#get the target addresses from the hospitals dataframe
target_index1 = 176
target_index2 = 173

target_address1 = hospitals['Address Line 1'][target_index1] + ', Cambridge'
target_address2 = hospitals['Address Line 1'][target_index2] + ', Cambridge'

#convert the addresses to lat long coordinates and get the nearest node on the graph
target_coords1 = ox.geocode(target_address1)
target_node1 = ox.nearest_nodes(G, target_coords1[1],target_coords1[0])

target_coords2 = ox.geocode(target_address2)
target_node2 = ox.nearest_nodes(G, target_coords2[1],target_coords2[0])

list_of_target_nodes = [target_node1, target_node2]

In [ ]:
list_of_target_nodes[0]

In [ ]:
#generate the random sample of 100 samples by first removing the target node location from the dataframe
nodes_df = nodes.copy()
nodes_df = nodes_df[~nodes_df.index.isin(list_of_target_nodes)] #get the dataframe of nodes that do not have any target nodes in them
nodes_sample = nodes_df[['y', 'x']].sample(n = 100, random_state = 1234)
nodes_sample

In [ ]:
#creating a function to calculate a score from a list of lengths calculated from the target node to each of the 100 sample nodes
def create_score(list_of_lengths):
    score = 1000
    for l in list_of_lengths:
        deduction = (((l/1000)/4.5)*60) * 5 #get the length in km divide by speed 4.5 km/h then divide by 60 to get time in minutes
        score = score - deduction #decrement the score by the derivation of time taken to each of the 100 nodes
        return score

In [ ]:
#define a function to calculate multiple shortest route lengths from the target node to each of the 100 sample nodes
def create_list_of_lengths(nodes_sample, target_node):
    list_of_lengths = []
    for node in nodes_sample.index:
        length = nx.shortest_path_length(G, source=node, target=target_node, weight='length') #calculate route from target node to sample node
        list_of_lengths.append(length) #append the length to the list
    
    return list_of_lengths
    

In [ ]:
#run the function defined above to generate a list of lengths between the target nodes and sample nodes
#then generate the score for each and store them in a dictionary
target_scores = {}
target_scores['Site 1'] = create_score(create_list_of_lengths(nodes_sample, list_of_target_nodes[0]))
target_scores['Site 2'] = create_score(create_list_of_lengths(nodes_sample, list_of_target_nodes[1]))
target_scores

In [ ]:
#print the scores for each proposed target site
print('The score for Site 1: {} is {}'.format(hospitals['Name'].iloc[target_index1], target_scores['Site 1']))
print('The score for Site 2: {} is {}'.format(hospitals['Name'].iloc[target_index2], target_scores['Site 2']))

In [ ]:
#initiate dictionary to store routes for all 100 nodes for each proposed site
target_to_nodes_routes = {}
target_node_names = ['Site1', 'Site2']

In [ ]:
#generate multiple shortest routes for each site and store in dictionary for plotting
for site, target_node in zip(target_node_names, list_of_target_nodes):
    list_of_routes = []
    for node in nodes_sample.index:
        route = nx.shortest_path(G, source=node, target=target_node, weight='length') #calculate route from target node to sample node
        list_of_routes.append(route) #append the length to the list
    target_to_nodes_routes[site] = list_of_routes
    


In [ ]:
import folium

In [ ]:
route_map = ox.plot_route_folium(G, target_to_nodes_routes[site][0], route_color = '#ff0000', opacity = 0.5)
for route in target_to_nodes_routes[site][1:len(target_to_nodes_routes[site])]:
    route_map = ox.plot_route_folium(G, route, route_map = route_map, route_color = '#ff0000', opacity = 0.5)
iframe = folium.IFrame('<font face = "Arial"><b>{}:</b> {}. <br><br><b>{} Score:</b> {}</br></br></font>'.format(site, target_address1, site, target_scores['Site 1']))
popup = folium.Popup(iframe, min_width=200, max_width=200)
folium.Marker(location=target_coords1,
popup = popup).add_to(route_map)

In [ ]:
m = folium.Map(location=target_coords2,tiles="cartodbpositron",zoom_start=13)

# layers to toggle
layer1 = folium.FeatureGroup(name='Route 1')
layer2 = folium.FeatureGroup(name='Route 2')
# create the route layers and add them to the map
route1 = [] 
for route in target_to_nodes_routes['Site1']:
    route_coords = []
    for node in route:
        route_coords.append((nodes.at[node, 'y'], nodes.at[node, 'x']))
        route1.append(route_coords)

route2 = []
for route in target_to_nodes_routes['Site2']:
    route_coords = []
    for node in route:
        route_coords.append((nodes.at[node, 'y'], nodes.at[node, 'x']))
        route2.append(route_coords)

folium.PolyLine(route1, color='red', weight=2, opacity=0.5).add_to(layer1)
folium.PolyLine(route2, color='green', weight=2, opacity=0.5).add_to(layer2)

# add the layers to the map
layer1.add_to(m)
layer2.add_to(m)

# add a layer control to toggle the layers on and off
folium.LayerControl().add_to(m)

# add a checkbox to toggle the layers on and off
folium.map.Marker(
[target_coords1[0], target_coords1[1]],
icon=folium.Icon(color='red', icon='info-sign')
).add_to(layer1)

folium.map.Marker(
[target_coords2[0], target_coords2[1]],
icon=folium.Icon(color='green', icon='info-sign')
).add_to(layer2)

site1_marker = folium.map.Marker(
[target_coords1[0], target_coords1[1]],
icon=folium.Icon(color='red', icon='info-sign')
).add_to(layer1)
site1_popup = folium.Popup('<b>Site 1:</b>: {}<br><b>Score:</b> {}'.format(target_address1,target_scores['Site 1']), max_width=250)
popup = folium.Popup(iframe, min_width=200, max_width=200)
site1_marker.add_child(site1_popup)
# add a marker and popup for each site
site2_marker = folium.map.Marker(
[target_coords2[0], target_coords2[1]],
icon=folium.Icon(color='green', icon='info-sign')
).add_to(layer2)
site2_popup = folium.Popup('<b>Site 2:</b> {}<br><b>Score:</b> {}'.format(target_address2,target_scores['Site 2']), max_width=250)
site2_marker.add_child(site2_popup)

m

In [ ]:
m